In [1]:
import pandas as pd
import numpy as np
import itertools
from statsmodels.tsa.statespace.sarimax import SARIMAX



train_df = pd.read_csv('./data/train_data.csv')
train_df['일시'] = pd.to_datetime(train_df['일시'])
endog_variable = '평균기온'
train_df = train_df[endog_variable]


In [ ]:

# Parameter search
p = range(0, 2)
d = range(0, 2)
q = range(0, 2)
pdq = list(itertools.product(p, d, q))

seasonal_pdq = [(x[0], x[1], x[2], 90) for x in list(itertools.product(p, d, q))]

best_aic = np.inf
best_pdq = None
best_seasonal_pdq = None
tmp_model = None
best_mdl = None

for param in pdq :
    for param_seasonal in seasonal_pdq :
        tmp_mdl = SARIMAX(train_df, exog=None, order = param,
                            seasonal_order = param_seasonal,
                            enforce_stationarity=True,
                            enforce_invertibility=True)
        res = tmp_mdl.fit()
        print("SARIMAX{}x{}12 - AIC:{}".format(param, param_seasonal, res.aic))
        if res.aic < best_aic :
              best_alc = res.aic
              best_pdq = param
              best_seasonal_pdq = param_seasonal
              best_mdl = tmp_mdl

print("Best SARIMAX{}x{}12 model - AIC:{}".format(best_pdq, best_seasonal_pdq, best_aic))

In [2]:
model =SARIMAX(endog=train_df, order =(2,1,2),
              seasonal_order=(1, 2, 2, 90),
              enforce_stationarity=True,
              enforce_invertibility=True)
model_fit = model.fit()
model_fit.summary()

/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/statespace/sarimax.py:1009: UserWarning: Non-invertible starting seasonal moving average Using zeros as starting parameters.
  warn('Non-invertible starting seasonal moving average'


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            8     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.88543D+00    |proj g|=  2.09775D-01


 This problem is unconstrained.


KeyboardInterrupt: 

In [ ]:
submission_df = pd.read_csv('./data/sample_submission.csv')

submission_df['일시'] = pd.to_datetime(submission_df['일시'])
submission_df.set_index('일시')
print(submission_df)



start_date = submission_df['일시'].min()
end_date = submission_df['일시'].max()

forecast = model_fit.predict(start=1, end=358, dynamic=True)

submission_df['평균기온'] = forecast.values
display(submission_df.head())

submission_df.to_csv('./SARIMAX_test.csv', index=False)